# 🧪 Exemplo de Aplicação com Conexão a Banco de Dados

Este notebook demonstra como criar uma aplicação simples em Python que interage com um banco de dados **PostgreSQL** utilizando bibliotecas como **Pandas**, **SQLAlchemy**, **Panel**, entre outras. A interface gráfica permite consultar, inserir, atualizar e excluir registros da tabela `pessoa`.

---

## 🛠️ Organização do Projeto e Ambiente Virtual

Para garantir isolamento e facilitar a manutenção do ambiente Python, é **fortemente recomendado** utilizar um **ambiente virtual**. Isso evita conflitos entre dependências de diferentes projetos.

### ✅ Criar ambiente virtual (Linux, macOS ou WSL)

```bash
python3 -m venv venv
source venv/bin/activate
```

### ✅ Criar ambiente virtual (Windows)

```bash
python -m venv venv
venv\Scripts\activate
```

---

## 📦 `requirements.txt` — Instalação de Dependências

Crie um arquivo chamado `requirements.txt` no diretório do projeto com o seguinte conteúdo:

```txt
pandas
sqlalchemy
psycopg2-binary
panel
python-dotenv
```

### ✅ Instalar as dependências com o pip

```bash
pip install -r requirements.txt
```

---

## 🔐 Utilizando o `.env` para Conexão com o Banco de Dados

Para proteger informações sensíveis como usuário, senha e nome do banco, recomendamos armazenar esses dados em um **arquivo `.env`**, que não deve ser incluído no repositório de código (como o GitHub).

### ✅ Exemplo de conteúdo do arquivo `.env`

```dotenv
DB_HOST=localhost
DB_NAME=fbd-conexao
DB_USER=postgres
DB_PASS=root
```

---

## 📎 `.env.example`: Informando a Estrutura Esperada

Crie também um arquivo chamado **`.env.example`**, que serve como modelo para outras pessoas saberem quais variáveis são esperadas no projeto (sem conter dados reais).

Esse arquivo **pode ser incluído no repositório**, pois não contém credenciais, apenas a estrutura necessária.

---

## 🚫 Protegendo Dados com `.gitignore`

Adicione os seguintes itens no seu arquivo `.gitignore` para evitar subir arquivos sensíveis ao repositório:

---

## 🧑‍💻 Rodando a Aplicação

Após configurar o banco de dados, instalar as dependências e ativar o ambiente virtual, você poderá executar a aplicação com:

```bash
panel serve nome_do_arquivo.py --autoreload --show
```

Ou, se estiver usando Jupyter Notebook, poderá importar as funções diretamente e utilizar a interface com `pn.panel(...)`.

---

In [ ]:
# Importa as bibliotecas
# Validar o mínimo de CPF
# Criar uma home direcionando para o crud e para o gráfico

import os
from dotenv import load_dotenv

import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn
import matplotlib.pyplot as plt

In [ ]:
# Carrega as variáveis do arquivo .env

load_dotenv()

In [ ]:
# Lê as variáveis de ambiente

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [ ]:
# Cria conexão com psycopg2 usando as variáveis carregadas

con = pg.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [ ]:
# Define a string de conexão para o SQLAlchemy, utilizando as variáveis do .env
# Cria o objeto engine do SQLAlchemy que será usado para conectar e executar comandos no banco

cnx_string = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'
engine = sqlalchemy.create_engine(cnx_string)

In [ ]:
# Executa a consulta SQL para buscar todos os 
# registros da tabela 'pessoa' no banco PostgreSQL 
# e carrega o resultado em um DataFrame do pandas


query = "select * from usuario;" 
df = pd.read_sql_query(query, engine)

df.head()

In [ ]:
# Inicializa as extensões do Panel necessárias:
# - Tabulator para tabelas interativas
# - Notificações na interface

pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

In [ ]:
# Criação da Home sem conectar os cliques ainda
btn_go_crud = pn.widgets.Button(name="📋 Ir para CRUD", button_type="primary", width=250)
btn_go_graficos = pn.widgets.Button(name="📊 Ir para Gráficos", button_type="success", width=250)
# btn_voltar_home = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')

In [ ]:
# Container principal da aplicação
main_area = pn.Column()

def create_home_view():
    home = pn.Column(
        pn.pane.Markdown("## 🏠 Tela Inicial"),
        pn.pane.Markdown("Escolha uma opção abaixo:"),
        pn.Spacer(height=20),
        btn_go_crud,
        btn_go_graficos,
        btn_go_campanhas,
        align="center",
        sizing_mode="stretch_width",
        margin=(50,50,50,50)
    )
    return home

In [ ]:
# Criação dos botões da Home
btn_go_crud = pn.widgets.Button(name="📋 Ir para CRUD", button_type="primary", width=250)
btn_go_graficos = pn.widgets.Button(name="📊 Ir para Gráficos", button_type="success", width=250)
btn_go_campanhas = pn.widgets.Button(name="📢 Ir para Campanhas", button_type="warning", width=250)


In [ ]:
# Funções de navegação
def go_to_home():
    main_area.clear()
    main_area.append(create_home_view())

In [ ]:
def create_btn_voltar():
    btn_voltar_home = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')
    btn_voltar_home.on_click(lambda e: main_area.clear() or main_area.append(create_home_view()))
    return btn_voltar_home

In [ ]:
# Variável auxiliar para consultas sem filtro
flag = ''

# Widget de seleção de tipo de pessoa
tipo_pessoa_widget = pn.widgets.RadioBoxGroup(
    name="Tipo de Pessoa",
    options=["Doador", "Beneficiário", "Instituição"],
    inline=False
)

# Widgets de entrada de dados
def create_form_widgets():
    return {
        "tipo_pessoa": tipo_pessoa_widget,
        "nome": pn.widgets.TextInput(
            name="Nome",
            placeholder="Digite o nome",
            sizing_mode="stretch_width"
        ),
        "cpf_cnpj": pn.widgets.TextInput(
            name="CPF/CNPJ",
            placeholder="Digite o CPF ou CNPJ",
            sizing_mode="stretch_width"
        ),
        "email": pn.widgets.TextInput(
            name="Email",
            placeholder="Digite o email",
            sizing_mode="stretch_width"
        ),
        "celular": pn.widgets.TextInput(
            name="Celular",
            placeholder="(XX) XXXXX-XXXX",
            sizing_mode="stretch_width"
        ),
        "senha": pn.widgets.PasswordInput(
            name="Senha",
            placeholder="Digite uma senha segura",
            sizing_mode="stretch_width"
        ),
        "datanasc": pn.widgets.DatePicker(
            name="Data de Nascimento",
            visible=False
        ),
        "rua": pn.widgets.TextInput(
            name="Rua",
            placeholder="Digite a rua",
            sizing_mode="stretch_width"
        ),
        "numero": pn.widgets.TextInput(
            name="Número",
            placeholder="Digite o número",
            sizing_mode="stretch_width"
        ),
        "bairro": pn.widgets.TextInput(
            name="Bairro",
            placeholder="Digite o bairro",
            sizing_mode="stretch_width"
        ),
        "cidade": pn.widgets.TextInput(
            name="Cidade",
            placeholder="Digite a cidade",
            sizing_mode="stretch_width"
        ),
        "estado": pn.widgets.TextInput(
            name="Estado",
            placeholder="Digite o estado",
            sizing_mode="stretch_width"
        ),
        "cep": pn.widgets.TextInput(
            name="CEP",
            placeholder="XXXXXXXX",
            sizing_mode="stretch_width"
        )
    }



In [ ]:
# Botões de ações CRUD
buttonConsultar = pn.widgets.Button(name='Consultar', button_type='primary')
buttonInserir = pn.widgets.Button(name='Inserir', button_type='success')
buttonExcluir = pn.widgets.Button(name='Excluir', button_type='danger')
buttonAtualizar = pn.widgets.Button(name='Atualizar', button_type='warning')

# Botão para voltar à Home (na tela CRUD)
buttonVoltar = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')


In [ ]:
# Funções CRUD adaptadas para a nova estrutura

def queryAll():
    """
    Consulta todos os usuários e retorna um Tabulator.
    """
    query = """
    SELECT u.cpf_cnpj, u.nome, u.email, u.celular, 
           CASE 
               WHEN d.cpf_cnpj_d IS NOT NULL THEN 'Doador'
               WHEN b.cpf_cnpj_b IS NOT NULL THEN 'Beneficiário'
               WHEN i.cpf_cnpj_i IS NOT NULL THEN 'Instituição'
           END AS tipo
    FROM usuario u
    LEFT JOIN doador d ON u.cpf_cnpj = d.cpf_cnpj_d
    LEFT JOIN beneficiario b ON u.cpf_cnpj = b.cpf_cnpj_b
    LEFT JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i
    ORDER BY u.nome
    """
    df = pd.read_sql_query(query, engine)
    return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)

def on_consultar(w):
    """Consulta pelo CPF/CNPJ ou retorna todos."""
    try:
        cpf_cnpj = w["cpf_cnpj"].value
        if cpf_cnpj.strip() == '':
            return queryAll()
        
        query = """
        SELECT u.cpf_cnpj, u.nome, u.email, u.celular, 
               CASE 
                   WHEN d.cpf_cnpj_d IS NOT NULL THEN 'Doador'
                   WHEN b.cpf_cnpj_b IS NOT NULL THEN 'Beneficiário'
                   WHEN i.cpf_cnpj_i IS NOT NULL THEN 'Instituição'
               END AS tipo
        FROM usuario u
        LEFT JOIN doador d ON u.cpf_cnpj = d.cpf_cnpj_d
        LEFT JOIN beneficiario b ON u.cpf_cnpj = b.cpf_cnpj_b
        LEFT JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i
        WHERE u.cpf_cnpj = %s
        """
        df = pd.read_sql_query(query, engine, params=(cpf_cnpj,))
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir(w):
    """Insere um novo usuário com seu tipo específico."""
    try:
        cursor = con.cursor()
        tipo = w["tipo_pessoa"].value
        cpf_cnpj = w["cpf_cnpj"].value
        
        if not cpf_cnpj or not w["nome"].value or not w["senha"].value:
            return pn.pane.Alert('CPF/CNPJ, Nome e Senha são obrigatórios!', alert_type='warning')
        
        # Insere na tabela usuário com senha
        cursor.execute(
            """INSERT INTO usuario(cpf_cnpj, nome, email, celular, rua, numero, bairro, 
                                   cidade, estado, cep, senha) 
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
            (cpf_cnpj, w["nome"].value, w["email"].value, w["celular"].value,
             w["rua"].value, w["numero"].value, w["bairro"].value, 
             w["cidade"].value, w["estado"].value, w["cep"].value, w["senha"].value)
        )
        
        # Insere na tabela específica
        if tipo == "Doador":
            cursor.execute(
                "INSERT INTO doador(cpf_cnpj_d, data_nascimento) VALUES (%s, %s)",
                (cpf_cnpj, w["datanasc"].value)
            )
        elif tipo == "Beneficiário":
            cursor.execute(
                "INSERT INTO beneficiario(cpf_cnpj_b, data_nascimento) VALUES (%s, %s)",
                (cpf_cnpj, w["datanasc"].value)
            )
        elif tipo == "Instituição":
            cursor.execute(
                "INSERT INTO instituicao(cpf_cnpj_i) VALUES (%s)",
                (cpf_cnpj,)
            )
        
        con.commit()
        cursor.close()
        pn.state.notifications.success('Usuário inserido com sucesso!')
        return queryAll()
    except Exception as e:
        con.rollback()
        cursor.close()
        return pn.pane.Alert(f'Erro ao inserir: {str(e)}', alert_type='danger')

def on_atualizar(w):
    """Atualiza dados do usuário."""
    try:
        cursor = con.cursor()
        cpf_cnpj = w["cpf_cnpj"].value
        
        if not cpf_cnpj:
            return pn.pane.Alert('CPF/CNPJ é obrigatório para atualizar!', alert_type='warning')
        
        # Se senha foi preenchida, atualiza também a senha
        if w["senha"].value:
            cursor.execute(
                """UPDATE usuario SET nome=%s, email=%s, celular=%s, rua=%s, numero=%s, 
                                   bairro=%s, cidade=%s, estado=%s, cep=%s, senha=%s
                   WHERE cpf_cnpj=%s""",
                (w["nome"].value, w["email"].value, w["celular"].value,
                 w["rua"].value, w["numero"].value, w["bairro"].value,
                 w["cidade"].value, w["estado"].value, w["cep"].value, w["senha"].value, cpf_cnpj)
            )
        else:
            cursor.execute(
                """UPDATE usuario SET nome=%s, email=%s, celular=%s, rua=%s, numero=%s, 
                                   bairro=%s, cidade=%s, estado=%s, cep=%s
                   WHERE cpf_cnpj=%s""",
                (w["nome"].value, w["email"].value, w["celular"].value,
                 w["rua"].value, w["numero"].value, w["bairro"].value,
                 w["cidade"].value, w["estado"].value, w["cep"].value, cpf_cnpj)
            )
        
        con.commit()
        cursor.close()
        pn.state.notifications.success('Usuário atualizado com sucesso!')
        return queryAll()
    except Exception as e:
        con.rollback()
        cursor.close()
        return pn.pane.Alert(f'Erro ao atualizar: {str(e)}', alert_type='danger')

def on_excluir(w):
    """Exclui um usuário e suas referências."""
    try:
        cursor = con.cursor()
        cpf_cnpj = w["cpf_cnpj"].value
        
        if not cpf_cnpj:
            return pn.pane.Alert('CPF/CNPJ é obrigatório para excluir!', alert_type='warning')
        
        # Deleta das tabelas específicas
        cursor.execute("DELETE FROM doador WHERE cpf_cnpj_d=%s", (cpf_cnpj,))
        cursor.execute("DELETE FROM beneficiario WHERE cpf_cnpj_b=%s", (cpf_cnpj,))
        cursor.execute("DELETE FROM instituicao WHERE cpf_cnpj_i=%s", (cpf_cnpj,))
        
        # Deleta da tabela usuário
        cursor.execute("DELETE FROM usuario WHERE cpf_cnpj=%s", (cpf_cnpj,))
        
        con.commit()
        cursor.close()
        pn.state.notifications.success('Usuário excluído com sucesso!')
        return queryAll()
    except Exception as e:
        con.rollback()
        cursor.close()
        return pn.pane.Alert(f'Erro ao excluir: {str(e)}', alert_type='danger')


In [ ]:
def table_creator(cons, ins, atu, exc, w):
    """
    Recebe booleans dos botões e executa a ação correspondente.
    """
    if cons: 
        return on_consultar(w)
    if ins: 
        resultado = on_inserir(w)
        # Reseta formulário após sucesso
        if isinstance(resultado, pn.widgets.Tabulator):
            for key in w:
                if hasattr(w[key], 'value'):
                    w[key].value = "" if key != "datanasc" else None
        return resultado
    if atu: 
        return on_atualizar(w)
    if exc: 
        return on_excluir(w)


In [ ]:
# Cria uma ligação interativa (bind) entre os botões e a função que executa a ação correspondente,
# atualizando a tabela na interface sempre que algum botão for clicado.

# Conecta os botões à tabela interativa
form_widgets = create_form_widgets()

def create_interactive_table():
    return pn.bind(
        table_creator,
        buttonConsultar,
        buttonInserir,
        buttonAtualizar,
        buttonExcluir,
        form_widgets
    )


In [ ]:
def create_crud_view(w):
    """
    Cria a tela do CRUD com layout bonito.
    """
    crud_layout = pn.Row(
        pn.Column(
            pn.pane.Markdown("### 👥 Gerenciador de Usuários"),
            w["tipo_pessoa"],
            pn.Spacer(height=10),
            w["nome"],
            w["cpf_cnpj"],
            w["email"],
            w["celular"],
            w["senha"],
            w["datanasc"],
            pn.Spacer(height=10),
            pn.pane.Markdown("**Endereço**"),
            w["rua"],
            w["numero"],
            w["bairro"],
            w["cidade"],
            w["estado"],
            w["cep"],
            pn.Spacer(height=15),
            pn.Row(buttonConsultar, buttonInserir),
            pn.Row(buttonAtualizar, buttonExcluir),
            pn.Spacer(height=10),
            create_btn_voltar(),
            sizing_mode='stretch_width',
            margin=(20, 20, 20, 20)
        ),
        pn.Column(create_interactive_table(), sizing_mode='stretch_both')
    )
    
    return crud_layout


In [ ]:
def go_to_crud():
    main_area.clear()
    main_area.append(create_crud_view(form_widgets))

In [ ]:
# Tela de Gráficos

def grafico_distribuicao_por_tipo():
    """Gráfico de pizza mostrando distribuição de tipos de usuários"""
    query = """
    SELECT 
        CASE 
            WHEN d.cpf_cnpj_d IS NOT NULL THEN 'Doador'
            WHEN b.cpf_cnpj_b IS NOT NULL THEN 'Beneficiário'
            WHEN i.cpf_cnpj_i IS NOT NULL THEN 'Instituição'
        END AS tipo,
        COUNT(*) AS quantidade
    FROM usuario u
    LEFT JOIN doador d ON u.cpf_cnpj = d.cpf_cnpj_d
    LEFT JOIN beneficiario b ON u.cpf_cnpj = b.cpf_cnpj_b
    LEFT JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i
    GROUP BY tipo
    """
    try:
        df = pd.read_sql_query(query, engine)
        fig, ax = plt.subplots(figsize=(8, 6))
        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
        ax.pie(df["quantidade"], labels=df["tipo"], autopct='%1.1f%%', colors=colors, startangle=90)
        ax.set_title("Distribuição de Usuários por Tipo")
        return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')
    except Exception as e:
        return pn.pane.Alert(f'Erro ao gerar gráfico: {str(e)}', alert_type='danger')

def grafico_instituicoes():
    """Gráfico de barras mostrando instituições cadastradas"""
    query = """
    SELECT u.nome, COUNT(c.id_campanha) AS campanhas
    FROM usuario u
    LEFT JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i
    LEFT JOIN campanha c ON i.cpf_cnpj_i = c.cpf_cnpj_i
    WHERE i.cpf_cnpj_i IS NOT NULL
    GROUP BY u.nome
    ORDER BY campanhas DESC
    """
    try:
        df = pd.read_sql_query(query, engine)
        if df.empty:
            return pn.pane.Alert('Nenhuma instituição cadastrada', alert_type='info')
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.barh(df["nome"], df["campanhas"], color='#17becf')
        ax.set_xlabel("Número de Campanhas")
        ax.set_title("Campanhas por Instituição")
        plt.tight_layout()
        return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')
    except Exception as e:
        return pn.pane.Alert(f'Erro ao gerar gráfico: {str(e)}', alert_type='danger')

def grafico_total_usuarios():
    """Gráfico mostrando total de usuários por tipo"""
    query = """
    SELECT 
        'Total de Usuários' AS categoria,
        COUNT(*) AS total
    FROM usuario
    """
    try:
        df = pd.read_sql_query(query, engine)
        fig, ax = plt.subplots(figsize=(8, 5))
        ax.bar(df["categoria"], df["total"], color='#FF6B6B', width=0.5)
        ax.set_ylabel("Quantidade")
        ax.set_title("Total de Usuários Cadastrados")
        for i, v in enumerate(df["total"]):
            ax.text(i, v + 1, str(v), ha='center', va='bottom', fontsize=14, fontweight='bold')
        return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')
    except Exception as e:
        return pn.pane.Alert(f'Erro ao gerar gráfico: {str(e)}', alert_type='danger')


In [ ]:
def create_graficos_view():
    graficos_layout = pn.Column(
        pn.pane.Markdown("### 📊 Análise do Sistema de Doações"),
        pn.Row(
            pn.Column(grafico_total_usuarios(), sizing_mode='stretch_width'),
            pn.Column(grafico_distribuicao_por_tipo(), sizing_mode='stretch_width')
        ),
        pn.Column(grafico_instituicoes(), sizing_mode='stretch_width'),
        create_btn_voltar(),
        sizing_mode='stretch_width',
        margin=(20,20,20,20)
    )
    
    return graficos_layout


In [ ]:
def go_to_graficos():
    main_area.clear()
    main_area.append(create_graficos_view())

In [ ]:
# ===== TELA DE CAMPANHAS =====

def get_instituicoes():
    """Busca todas as instituições cadastradas"""
    try:
        query = """
        SELECT DISTINCT u.cpf_cnpj, u.nome
        FROM usuario u
        JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i
        ORDER BY u.nome
        """
        df = pd.read_sql_query(query, engine)
        return dict(zip(df['nome'], df['cpf_cnpj']))
    except Exception as e:
        return {}

# Widgets para campanhas
instituicoes_dict = get_instituicoes()
instituicoes_select = pn.widgets.Select(
    name="Instituição",
    options=instituicoes_dict,
    sizing_mode="stretch_width"
)

nome_campanha = pn.widgets.TextInput(
    name="Nome da Campanha",
    placeholder="Digite o nome da campanha",
    sizing_mode="stretch_width"
)

data_inicio = pn.widgets.DatePicker(
    name="Data de Início",
    sizing_mode="stretch_width"
)

data_fim = pn.widgets.DatePicker(
    name="Data de Término",
    sizing_mode="stretch_width"
)

# Botões de ações para campanhas
btn_criar_campanha = pn.widgets.Button(name='Criar Campanha', button_type='success')
btn_consultar_campanhas = pn.widgets.Button(name='Consultar Campanhas', button_type='primary')

def on_criar_campanha(event):
    """Cria uma nova campanha"""
    try:
        if not instituicoes_select.value or not nome_campanha.value:
            return pn.pane.Alert('Instituição e Nome são obrigatórios!', alert_type='warning')
        
        cursor = con.cursor()
        cursor.execute(
            """INSERT INTO campanha(cpf_cnpj_i, nome, data_inicio, data_fim, status) 
               VALUES (%s, %s, %s, %s, %s)""",
            (instituicoes_select.value, nome_campanha.value,
             data_inicio.value, data_fim.value, 'Planejada')
        )
        con.commit()
        cursor.close()
        pn.state.notifications.success('Campanha criada com sucesso!')
        
        # Limpar campos
        nome_campanha.value = ""
        data_inicio.value = None
        data_fim.value = None
        
    except Exception as e:
        con.rollback()
        cursor.close()
        return pn.pane.Alert(f'Erro ao criar campanha: {str(e)}', alert_type='danger')

def on_consultar_campanhas(event):
    """Consulta todas as campanhas"""
    try:
        query = """
        SELECT c.id_campanha, u.nome AS instituicao, c.nome, 
               c.data_inicio, c.data_fim, c.status
        FROM campanha c
        JOIN instituicao i ON c.cpf_cnpj_i = i.cpf_cnpj_i
        JOIN usuario u ON i.cpf_cnpj_i = u.cpf_cnpj
        ORDER BY c.data_inicio DESC
        """
        df = pd.read_sql_query(query, engine)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro ao consultar: {str(e)}', alert_type='danger')

btn_criar_campanha.on_click(on_criar_campanha)

def create_campanhas_view():
    """Cria a tela de campanhas"""
    campanhas_layout = pn.Column(
        pn.pane.Markdown("### 📢 Gerenciador de Campanhas"),
        pn.pane.Markdown("**Criar Nova Campanha**"),
        instituicoes_select,
        nome_campanha,
        data_inicio,
        data_fim,
        pn.Row(btn_criar_campanha, btn_consultar_campanhas),
        pn.Spacer(height=20),
        pn.bind(on_consultar_campanhas, btn_consultar_campanhas),
        create_btn_voltar(),
        sizing_mode='stretch_width',
        margin=(20, 20, 20, 20)
    )
    return campanhas_layout

def go_to_campanhas():
    main_area.clear()
    main_area.append(create_campanhas_view())

In [ ]:
# Conecta os botões da Home às funções de navegação
btn_go_crud.on_click(lambda event: go_to_crud())
btn_go_graficos.on_click(lambda event: go_to_graficos())
btn_go_campanhas.on_click(lambda event: go_to_campanhas())


In [ ]:
# Inicializa com a tela Home
go_to_home()

In [ ]:
# App principal, com título, menu superior e área dinâmica
# App principal, com título e área dinâmica
pn.Column(
    pn.pane.Markdown("# 🎁 Sistema de Doações"),
    main_area
).servable()